# Data Importing

In [1]:
# Importing modules
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression

# Read data into papers
data = pd.read_csv("/Users/atchoo/Google Drive/Research Assistant/Work With David Knight/Survey of the Incarcerated/TMPPoliticalSurveyFULL_ForDavid.csv", encoding = "ISO-8859-1")

# Print head
# print(data.columns)

In [2]:
# Insert an id index for future work
data["Survey_ID"] = data.index + 1

# Remove the columns
str_data = data[["Survey_ID", "explain_politics_changed_since_incarcerated",\
                 "explain_race_affects_politics", 'identifies_as_black', 'identifies_as_white',
                 "identifies_as_native", 'identifies_as_asian', 
                 'identifies_as_hawaiian_or_pac_islander', 'identifies_as_other_race',
                 'identifies_as_not_sure_of_race', 'identifies_as_hispanic_or_latinx',
                 "age", "gender"]]
# Print out the first rows of papers
str_data.columns = ["Survey_ID", "p_change", "r_effect", "black", "white", "native", "asian",
                   "hawaiian", "other_race", "unsure_race", "latinx", "age", "gender"]
str_data = str_data.dropna()

# Make Remove Stopword Functions

In [3]:
# Load the regular expression library and the nltk word library
import re
import nltk

# Create a function to remove nonsense words
words = set(nltk.corpus.words.words())
def clean_sent(sent):
    return " ".join(w for w in nltk.wordpunct_tokenize(sent) \
     if w.lower() in words or not w.isalpha())

strv_list = ["p_change", "r_effect"]
for strv in strv_list:
    # Remove punctuation & Convert the titles to lowercase
    str_data[strv] = str_data[strv].map(lambda x: re.sub(r'[^\w\s]', "", x)).map(lambda x: x.lower())
    # Remove nonsense words
    str_data[strv] = str_data[strv].apply(clean_sent)
    # Transform blank cells to NaN & Drop NaN
    str_data = str_data.replace(r'^\s*$', np.nan, regex=True).dropna()

In [4]:
str_data.head()

,Survey_ID,p_change,r_effect,black,white,native,asian,hawaiian,other_race,unsure_race,latinx,age,gender
0,1,no,not at all not racist all of the human race is...,False,True,False,False,False,False,False,False,46-55,Man
2,3,i feel that the parole board the some of the p...,not sure about that,False,True,False,False,False,False,False,False,46-55,Woman
3,4,yes bit,i am not really sure about that one,False,True,False,False,False,False,False,False,36-45,Woman
5,6,most are full of,i do not understand this question maybe theres...,True,False,False,False,False,False,False,False,26-35,Woman
6,7,i never considered the prison population or pr...,i try to vote based on the but i am and many i...,False,True,False,False,False,False,False,False,46-55,Woman


In [5]:
# remove stop words
# nltk.download('stopwords')
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Data Exporting for C++

In [6]:
extrastopword = []

In [7]:
stop_words = stopwords.words("english")
stop_words.extend(["don", "people", "bill", "step", "act", "first", "u", "n", 
                   "na", "non"]+extrastopword)
yesnno = ["yes", "no"]
stop_words =  [word for word in stop_words if word not in yesnno]

In [8]:
r_effect = remove_stopwords(str_data["r_effect"])
r_effect = pd.DataFrame([x for x in r_effect if x])
len(r_effect)

3095

In [9]:
#r_effect.to_csv('/Users/atchoo/Desktop/r_effect_3082.txt', header=None, index=None, sep=' ', mode='a')

# Include the Topic info

In [10]:
def add_the_matrix(extrastopword, csvfile, tlist=["topic 5"]):
    
    assert isinstance(extrastopword, list), "Should be a list of the extra stop words."
    assert isinstance(csvfile, str), "Should be the path of the matrix file."
    
    # change the stop words
    stop_words = stopwords.words("english")
    stop_words.extend(["don", "people", "bill", "step", "act", "first", "u", "n", 
                       "na", "non"]+extrastopword)
    yesnno = ["yes", "no"]
    stop_words =  [word for word in stop_words if word not in yesnno]
    # remove stop words
    r_effect = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in str_data["r_effect"]]
    r_effect = pd.DataFrame([x for x in r_effect if x])
    
    # create index variable for whether a row is left
    r_effect2 = []
    for doc in str_data["r_effect"]:
        if all(word in stop_words for word in simple_preprocess(doc)):
               i_str = False
        else: 
               i_str = True
        r_effect2.append(i_str)
    inBTM = pd.DataFrame(r_effect2, columns=["inBTM"])
    # filter str_data based on inBTM result into str_datar
    str_datar = pd.concat([str_data.reset_index(drop=True), inBTM], axis=1)
    str_datar = str_datar[str_datar["inBTM"]==True]
    
    # read in the topicmatrix file
    matrix = pd.read_csv(csvfile, 
                         names=["topic 1", "topic 2", "topic 3", "topic 4", "topic 5", "topic 6"])
    a = matrix.idxmax(axis=1)
    b = matrix.T.apply(lambda x: x.nlargest(2).idxmin())
    c = matrix.T.apply(lambda x: x.nlargest(3).idxmin())
    topic = pd.concat([a, b, c], axis=1, ignore_index=False)

    # concatenate topic matrix and str_datar as str_datar
    str_datar2 = pd.concat([str_datar.reset_index(drop=True), topic], axis=1)
    str_datar2 = str_datar2.rename(columns={0:"fsttopic", 1:"sndtopic", 2:"trdtopic"})

    # CREATE THE CHISQUARE DATAFRAME
    race = ["black", "white", "native", "asian", "hawaiian", "other_race", "unsure_race", "latinx"]
    topic = ["fsttopic", "sndtopic", "trdtopic"]
    chisq = str_datar2.loc[:, ["Survey_ID"]+race+topic].dropna()
        ## clean the race variable a little bit
    chisq.loc[:, race] = chisq.loc[:, race].astype(int)
    chisq["Race"] = chisq.loc[:, race].idxmax(axis=1)
    
    # CREATE THE REGRESSION DATAFRAME
    chisq["fst5"] = chisq["fsttopic"].isin(tlist)
    chisq["snd5"] = chisq["sndtopic"].isin(tlist)
    chisq["trd5"] = chisq["trdtopic"].isin(tlist)
    cor = chisq.drop(["fsttopic", "sndtopic", "trdtopic", "Race"], axis=1)
    cov = str_data[["Survey_ID", "age", "gender"]]
    cov = pd.get_dummies(cov, columns=["age", "gender"])
    lrdata = cor.merge(cov, on="Survey_ID")
    
    return lrdata

In [11]:
common_csv = "~/Google Drive/Research Assistant/Work With David Knight/Survey of the Incarcerated/BTM/BTM_Regression_Exploration/"
extra1 = []
csv1 = "All_3095/topicmatrix.csv"

extra2 = ["violent", "race"]
csv2 = "NoViloentRace_3082/topicmatrix_3082.csv"

extra3 = ["violent", "race", "white", "black"]
csv3 = "NoViloentRaceWhiteBlack_3053/topicmatrix_3053.csv"

In [12]:
lrdata4 = add_the_matrix(extra1, common_csv+csv1)
lrdata5 = add_the_matrix(extra2, common_csv+csv2)
lrdata6 = add_the_matrix(extra3, common_csv+csv3)

# Linear Regression

In [13]:
lrdata4.columns

Index(['Survey_ID', 'black', 'white', 'native', 'asian', 'hawaiian',
       'other_race', 'unsure_race', 'latinx', 'fst5', 'snd5', 'trd5',
       'age_18-25', 'age_26-35', 'age_36-45', 'age_46-55', 'age_56-65',
       'age_66+', 'age_Under 18',
       'gender_Gender non-conforming or non-binary or other', 'gender_Man',
       'gender_Prefer not to say', 'gender_Woman'],
      dtype='object')

In [14]:
def reg_on_123(dataset4lr, dropX=[]):
    y1 = dataset4lr["fst5"]
    y2 = dataset4lr["snd5"]
    y3 = dataset4lr["trd5"]

    X = dataset4lr.drop(["Survey_ID", "fst5", "snd5", "trd5", "unsure_race", "age_Under 18",
                         "gender_Gender non-conforming or non-binary or other"]+dropX, axis=1)
    X = sm.add_constant(X)
    
    model1 = sm.OLS(y1, X).fit()
    model2 = sm.OLS(y2, X).fit()
    model3 = sm.OLS(y3, X).fit()
    
    return model1, model2, model3

def return_plist(dataset4lr, dropX=[]):
    model1, model2, model3 = reg_on_123(dataset4lr, dropX)
    
    a = (model1.summary2().tables[1]['P>|t|'] <= 0.05) | (model1.summary2().tables[1]['P>|t|'] >= 0.95)
    a1 = model1.summary2().tables[1]["Coef."]
    b = (model2.summary2().tables[1]['P>|t|'] <= 0.05) | (model2.summary2().tables[1]['P>|t|'] >= 0.95)
    b1 = model2.summary2().tables[1]["Coef."]
    c = (model3.summary2().tables[1]['P>|t|'] <= 0.05) | (model3.summary2().tables[1]['P>|t|'] >= 0.95)
    c1 = model3.summary2().tables[1]["Coef."]
    
    p_df = pd.concat([a1, a, b1, b, c1, c], axis=1)
    p_df.columns = ["fst_inlist_coef", "p<=0.05", "snd_inlist_coef", "p<=0.05", "trd_inlist_coef", "p<=0.05"]
    
    return p_df

### Keep All
These groups of models only excluded some variables to clear perfect multicollinearity.
The variables excluded for perfect multicollinearity include "Survey_ID", "fst1n2", "snd1n2", "trd1n2", "unsure_race","age_Under 18", "gender_Gender non-conforming or non-binary or other".

In [15]:
# All_3095
S3095_all = return_plist(lrdata4)
# NoViloentRace_3082
S3082_all = return_plist(lrdata4)
# NoViloentRaceWhiteBlack_3053
S3053_all = return_plist(lrdata4)

/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [16]:
print(S3053_all)

                          fst_inlist_coef  p<=0.05  snd_inlist_coef  p<=0.05  \
const                           -0.053770    False         0.151685    False   
black                            0.026793    False        -0.002332    False   
white                           -0.004315    False        -0.019005    False   
native                           0.023902    False        -0.001953    False   
asian                           -0.029110    False         0.008141    False   
hawaiian                        -0.019929    False        -0.039173    False   
other_race                      -0.009233    False        -0.011365    False   
latinx                           0.041802     True        -0.056367     True   
age_18-25                        0.132390    False        -0.025742    False   
age_26-35                        0.161439    False        -0.035056    False   
age_36-45                        0.176766    False        -0.040463    False   
age_46-55                        0.19452

In [17]:
S3053_all.to_csv("S3053_all.csv")

### No White
Other than same variables above to clear perfect multicollinearity, these groups of models also excluded "white".

In [18]:
nw = ["white"]

In [19]:
# All_3095
S3095_nw = return_plist(lrdata4, nw)
# NoRace_3082
S3082_nw = return_plist(lrdata5, nw)
# NoRaceWhiteBlack_3053
S3053_nw = return_plist(lrdata6, nw)

In [20]:
print(S3053_nw)

                          fst_inlist_coef  p<=0.05  snd_inlist_coef  p<=0.05  \
const                            0.495672     True         0.116277    False   
black                           -0.006549    False        -0.024520    False   
native                          -0.013793    False         0.017271    False   
asian                           -0.077258    False        -0.001166     True   
hawaiian                         0.024950    False        -0.012745    False   
other_race                       0.003301    False        -0.005176    False   
latinx                          -0.023055    False        -0.028570    False   
age_18-25                       -0.169265    False         0.001167     True   
age_26-35                       -0.231025    False         0.055879    False   
age_36-45                       -0.230479    False         0.057419    False   
age_46-55                       -0.231769    False         0.048819    False   
age_56-65                       -0.24861

In [21]:
S3053_nw.to_csv("S3053_nw.csv")

### Only White
Other than same variables above to clear perfect multicollinearity, these groups of models also excluded all race binary variables excpet "white".

In [22]:
ow = ["black", "native", "asian", "hawaiian", "other_race", "unsure_race", "latinx"]

In [23]:
# All_3095
S3095_ow = return_plist(lrdata4, ow)
# NoRace_3082
S3082_ow = return_plist(lrdata5, ow)
# NoRaceWhiteBlack_3053
S3053_ow = return_plist(lrdata6, ow)

In [24]:
print(S3053_ow)

                          fst_inlist_coef  p<=0.05  snd_inlist_coef  p<=0.05  \
const                            0.466631     True         0.092894    False   
white                            0.027897    False         0.018992    False   
age_18-25                       -0.169665    False         0.002883     True   
age_26-35                       -0.231756    False         0.060152    False   
age_36-45                       -0.230465    False         0.062983    False   
age_46-55                       -0.231543    False         0.054391    False   
age_56-65                       -0.247753    False         0.080931    False   
age_66+                         -0.190891    False         0.069769    False   
gender_Man                      -0.063853    False         0.059592    False   
gender_Prefer not to say        -0.167843    False         0.069451    False   
gender_Woman                    -0.002946     True         0.013799    False   

                          trd_inlist_co

In [25]:
S3053_ow.to_csv("S3053_ow.csv")

### No Black
Other than same variables above to clear perfect multicollinearity, these groups of models also excluded "black".

In [26]:
nb = ["black"]

In [27]:
# All_3095
S3095_nb = return_plist(lrdata4, nb)
# NoRace_3082
S3082_nb = return_plist(lrdata5, nb)
# NoRaceWhiteBlack_3053
S3053_nb = return_plist(lrdata6, nb)

In [28]:
print(S3053_nb)

                          fst_inlist_coef  p<=0.05  snd_inlist_coef  p<=0.05  \
const                            0.478389     True         0.098136    False   
white                            0.026706    False         0.016414    False   
native                          -0.011700    False         0.020837    False   
asian                           -0.076745    False        -0.002985     True   
hawaiian                         0.020909    False        -0.017263    False   
other_race                       0.011581    False         0.002052    False   
latinx                          -0.016674    False        -0.022438    False   
age_18-25                       -0.173659    False        -0.001372     True   
age_26-35                       -0.237061    False         0.054523    False   
age_36-45                       -0.236707    False         0.056165    False   
age_46-55                       -0.238351    False         0.047682    False   
age_56-65                       -0.25496

In [29]:
S3053_nb.to_csv("S3053_nb.csv")

### Only Black
Other than same variables above to clear perfect multicollinearity, these groups of models also excluded all race binary variables excpet "black".

In [30]:
ob = ["white", "native", "asian", "hawaiian", "other_race", "unsure_race", "latinx"]

In [31]:
# All_3095
S3095_ob = return_plist(lrdata4, ob)
# NoRace_3082
S3082_ob = return_plist(lrdata5, ob)
# NoRaceWhiteBlack_3053
S3053_ob = return_plist(lrdata6, ob)

In [32]:
print(S3053_ob)

                          fst_inlist_coef  p<=0.05  snd_inlist_coef  p<=0.05  \
const                            0.476932     True         0.104522    False   
black                           -0.003627    False        -0.022642    False   
age_18-25                       -0.163838    False         0.007520     True   
age_26-35                       -0.223436    False         0.064440    False   
age_36-45                       -0.221618    False         0.067497    False   
age_46-55                       -0.221157    False         0.059993    False   
age_56-65                       -0.237705    False         0.086595    False   
age_66+                         -0.179862    False         0.074981    False   
gender_Man                      -0.066044    False         0.059675    False   
gender_Prefer not to say        -0.175346    False         0.066267    False   
gender_Woman                    -0.001053     True         0.015257    False   

                          trd_inlist_co

In [33]:
S3053_ob.to_csv("S3053_ob.csv")